In [2]:
%runfile utils/matrix_factorizations.sage

In [3]:
def create_permutation_matrix(shift):
    s = copy(shift)
    m = len(s)
    A = list(list(zip(*sorted( [(s[i],i+1) for i in range(m)] )))[1])
    Mn = MatrixSpace(ZZ,m,m)
    
    I = Mn.identity_matrix()
    perm = Permutation(A)
    return perm

In [4]:
def Basis(F0,s = None):
    m,n = F0.dimensions()
    if (s == None):
        s = [0]*m

    pi = create_permutation_matrix(s)
    K,rrp, nrrp = PLUQ_to_left_kernel(F0.with_permuted_rows(pi))
    r = m - K.nrows()
    Mn = MatrixSpace(ZZ,m,m)
    I = Mn.identity_matrix()
    pi_mat = I.with_permuted_rows(pi)
    P = Permutation([i+1 for i in rrp+nrrp]).inverse() 
    P_mat =  I.with_permuted_columns(P)
    M = (P_mat*pi_mat).inverse_of_unit()*block_matrix([[x,0],[K,1]])*(P_mat*pi_mat)
    u = (P_mat*pi_mat).inverse_of_unit()*((P_mat*pi_mat)*vector(s) + vector([1]*r+[0]*(m-r)))
    return M,u

In [5]:
m = 16
n = 8
F = GF(97)
pR.<x> = F[]

In [6]:
M = MatrixSpace(F,m,n)
Mm = MatrixSpace(F,m,n)

MX = MatrixSpace(pR,m,m)

In [7]:
for i in range(100):
    A = M.random_element() 
    shift = (ZZ^m).random_element()
    B, new_shift= Basis(A,shift)
    if (not B.is_minimal_approximant_basis(A,1,shift)):
        print("error")
    if (list(new_shift) != B.row_degrees(shift)):
        print("wrong shift")

In [8]:
m = 8
n = 5 
F = GF(3)
sigma = 5

pR.<x> = F[]
Ms = MatrixSpace(pR,m,n)
Ms_n = MatrixSpace(pR,m,m)

In [9]:
def M_Basis(F,sigma,shift = None):
    pR = F.base_ring()
    x = pR.gen()
    m,n = F.dimensions()
    R = parent(F)
    if (shift == None):
        shift = [0]*m
        
    P = [0]*sigma
    u = [0]*sigma
    M = [0]*sigma
    P[0],u[0] = Basis(F.coefficient_matrix(0), shift)
    for k in range(1,sigma):
        F_prime = R(x^(-k)*P[k-1]*F).coefficient_matrix(0)
        M[k],u[k] = Basis(F_prime, u[k-1])
        P[k] = M[k]*P[k-1]
    return P[-1], u[-1]

In [12]:
for i in range(100):
    A = Ms.random_element() 
    sigma = ZZ.random_element(1,10)
    shift = (ZZ^m).random_element()
    B, new_shift= M_Basis(A, sigma, shift)
    if (not B.is_minimal_approximant_basis(A,sigma,shift)):
        print("error")

In [18]:
def PM_Basis(F, sigma, shifts = None):
    m,n = F.dimensions()
    if (shifts == None):
        shifts = [0]*m
        
    if sigma == 1:
        return Basis(F.coefficient_matrix(0), shifts)
    
    Pl, ul = PM_Basis(F, floor(sigma/2), shifts)
    F_prime = MP(Pl, F, floor(sigma/2) + 1, sigma)
    Ph, uh = PM_Basis(F_prime, ceil(sigma/2), ul)
    return Ph*Pl, uh

In [19]:
def MP(P, F, i, j):
    D = P*F
    R = parent(D)
    pR = D.base_ring()
    x = pR.gen()
    m,n = D.dimensions()
    for k in range(m):
        for l in range(n):
            P = D[k,l]
            P = P.shift(-(i-1)) % x^(j-i+1)
            D[k,l] = P
    return D

In [20]:
m = 8
n = 5 
F = GF(3)
sigma = 5

pR.<x> = F[]
Ms = MatrixSpace(pR,m,n)
Ms_n = MatrixSpace(pR,m,m)

In [21]:
for i in range(100):
    A = Ms.random_element() 
    sigma = ZZ.random_element(1,10)
    shift = (ZZ^m).random_element()
    B, new_shift= PM_Basis(A, sigma, shift)
    if (not B.is_minimal_approximant_basis(A,sigma,shift)):
        print("error")